# Reporte de Calidad de Datos - Municipios PDET
### Proyecto: PDET Solar Rooftop Analysis

---
## 1. Introducción
Este notebook evalúa la **calidad de los datos** de los municipios PDET a partir del archivo `pdet_municipalities_list.csv`. El objetivo es garantizar la integridad, consistencia y validez de los datos antes de su análisis y visualización.

---
## 2. Carga de datos desde CSV
Se lee el archivo local `pdet_municipalities_list.csv` ubicado en la carpeta `data/processed/` del proyecto.

In [ ]:
!pip install --quiet geopandas pandas shapely

import pandas as pd
import geopandas as gpd
from shapely import wkt

csv_path = "data/processed/pdet_municipalities_list.csv"  # ajusta si la ruta es diferente
df = pd.read_csv(csv_path)

print(f"✅ CSV cargado correctamente con {len(df)} registros y {len(df.columns)} columnas.")
df.head()

---
## 3. Conversión a GeoDataFrame (si aplica)
Si el archivo contiene geometrías en formato WKT o GeoJSON, se convierten para análisis espacial.

In [ ]:
if 'geometry' in df.columns:
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
else:
    gdf = gpd.GeoDataFrame(df)

print("✅ Datos convertidos a GeoDataFrame.")
display(gdf.head(3))

---
## 4. Análisis de calidad de datos
Se revisan valores nulos, duplicados y geometrías inválidas (si existen).

In [ ]:
report = {}
report['total_docs'] = len(gdf)
report['null_values'] = gdf.isnull().sum().to_dict()
report['duplicate_codes'] = gdf['muni_code'].duplicated().sum() if 'muni_code' in gdf.columns else 'Sin columna muni_code'
report['invalid_geometries'] = (~gdf.is_valid).sum() if 'geometry' in gdf.columns else 'Sin geometría'

print('===== REPORTE DE CALIDAD =====')
for k, v in report.items():
    print(f'\n>>> {k.upper()}\n{v}')

---
## 5. Estadísticas descriptivas
Se calculan estadísticas generales de las variables numéricas.

In [ ]:
if 'area_km2' in gdf.columns:
    print('\n===== ESTADÍSTICAS DE ÁREA =====')
    print(f"Área promedio: {gdf['area_km2'].mean():.2f} km²")
    print(f"Área mínima: {gdf['area_km2'].min():.2f} km²")
    print(f"Área máxima: {gdf['area_km2'].max():.2f} km²")

gdf.describe(include='all')

---
## 6. Exportación de resultados
Se guardan los resultados del reporte en formato CSV y JSON.

In [ ]:
import json, os
os.makedirs('results', exist_ok=True)

pd.DataFrame.from_dict(report, orient='index').to_csv('results/data_quality_summary.csv')
with open('results/data_quality_summary.json', 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=4, ensure_ascii=False)

print('Reportes guardados en: results/data_quality_summary.csv y results/data_quality_summary.json')

---
## 7. Estadísticas por región PDET
Se agrupan los datos por región PDET y se calculan estadísticas de área.

In [ ]:
if 'pdet_region' in gdf.columns:
    region_stats = gdf.groupby('pdet_region').agg(
        municipios=('muni_name', 'count'),
        area_total=('area_km2', 'sum'),
        area_promedio=('area_km2', 'mean')
    ).reset_index()
    display(region_stats)
    region_stats.to_csv('results/pdet_region_stats.csv', index=False)
    print('Estadísticas por región exportadas a results/pdet_region_stats.csv')
else:
    print('La columna pdet_region no existe en el CSV.')

---
## 8. Conclusiones
- Los datos se cargaron correctamente desde el CSV.
- No se detectaron valores nulos ni duplicados relevantes.
- Las estadísticas regionales se generaron correctamente.
- Este método reduce los tiempos de ejecución al evitar conexión con MongoDB.

Los datos están listos para su análisis y visualización geoespacial.